<a href="https://colab.research.google.com/github/sam2005-max/Anti-Risk/blob/main/Risk_model_MVP_telegram_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install telebot

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.1/260.1 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyTelegramBotAPI: filename=pyTelegramBotAPI-4.15.4-py3-none-any.whl size=231937 sha256=5fb7c6c8e669e55fd1a1996b11dd0f9487e058f9c2c3b10c87d270c30b681dd7
  Stored in directory: /root/.cache/pip/wheels/37/50/a0/68519c75209358c42758e5ed2d3c9da9afda4d20a3ac393070
Successfully built pyTelegramBotAPI


In [3]:
import telebot
from telebot import types
import pandas as pd
from io import BytesIO
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_validate

# Bu yerga o'zingizning bot tokeningizni qo'shing
bot_token = "6749312916:AAE2Y0i3Yqyn-N7HhOJXrXcmY9YcyR64bjE"

# Botni yaratish
bot = telebot.TeleBot(bot_token)


# /start buyrug'i
@bot.message_handler(commands=['start'])
def handle_start(message):
    user = message.from_user
    bot.send_message(user.id, f"Assalomu aleykum 🤗, {user.first_name}! Anti-Risk botimizga hush kelibsiz. Siz menga shaxsga doir ma'lumotlarni yuvoring va men esa sizga ushbu mijozning kredit holatini bashorat qilib yuvoraman")


# CSV fayl yuborilganda
@bot.message_handler(content_types=['document'])
def handle_csv(message):
    user = message.from_user
    file_info = bot.get_file(message.document.file_id)
    downloaded_file = bot.download_file(file_info.file_path)
    data1 = pd.read_csv('credit_risk_dataset.csv')

    # Data Cleaning Ma'lumotlarga ishlov berish
    data1['person_home_ownership'] = data1['person_home_ownership'].str.replace('RENT', '0')
    data1['person_home_ownership'] = data1['person_home_ownership'].str.replace('OWN', '1')
    data1['person_home_ownership'] = data1['person_home_ownership'].str.replace('MORTGAGE', '2')
    data1['person_home_ownership'] = data1['person_home_ownership'].str.replace('OTHER', '3')
    data1['person_home_ownership'] = data1['person_home_ownership'].astype(int)
    data1['loan_intent'] = data1['loan_intent'].str.replace('PERSONAL', '0')
    data1['loan_intent'] = data1['loan_intent'].str.replace('EDUCATION', '1')
    data1['loan_intent'] = data1['loan_intent'].str.replace('MEDICAL', '2')
    data1['loan_intent'] = data1['loan_intent'].str.replace('VENTURE', '3')
    data1['loan_intent'] = data1['loan_intent'].str.replace('HOMEIMPROVEMENT', '4')
    data1['loan_intent'] = data1['loan_intent'].str.replace('DEBTCONSOLIDATION', '5')
    data1['loan_intent'] = data1['loan_intent'].astype(int)
    data1['loan_grade'] = data1['loan_grade'].str.replace('D', '0')
    data1['loan_grade'] = data1['loan_grade'].str.replace('B', '1')
    data1['loan_grade'] = data1['loan_grade'].str.replace('C', '2')
    data1['loan_grade'] = data1['loan_grade'].str.replace('A', '3')
    data1['loan_grade'] = data1['loan_grade'].str.replace('E', '4')
    data1['loan_grade'] = data1['loan_grade'].str.replace('F', '5')
    data1['loan_grade'] = data1['loan_grade'].str.replace('G', '6')
    data1['loan_grade'] = data1['loan_grade'].astype(int)
    data1['cb_person_default_on_file'] = data1['cb_person_default_on_file'].str.replace('Y', '0')
    data1['cb_person_default_on_file'] = data1['cb_person_default_on_file'].str.replace('N', '1')
    data1['cb_person_default_on_file'] = data1['cb_person_default_on_file'].astype(int)

    # Ma'llumotlarni ichidagi yo'q (null) qiymatlarni to'ldirib chiqamiz 0 va o'rtacha (mean) qiymatlar bilan
    data1['loan_int_rate'].fillna(data1['loan_int_rate'].mean(), inplace=True)
    data1['person_emp_length'].fillna(0, inplace=True)

    # moljal qaratilgan ma'lumotni olamiz va keraksizlarini o'chirib tashlaymiz
    X = data1.drop(['loan_status', 'cb_person_default_on_file'], axis=1)
    y = data1['loan_status']

    # Ma'lumotlarni train qildirish uchun bo'lib olamiz
    X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=33)

    # model parametrlari
    model_rf = RandomForestClassifier(n_estimators=500, oob_score=True, n_jobs=-1,
                                      random_state=50, max_leaf_nodes=30)
    # ma'lumotlarni modelga moslash
    model_rf.fit(X_train, y_train)

    # bashorat
    prediction_test = model_rf.predict(X_test)

    # model bashoratining aniqligini chiqaramiz
    accuracy = accuracy_score(y_test, prediction_test)
    file_content = BytesIO(downloaded_file)

    # Use the file-like object to read the CSV
    for_test = pd.read_csv(file_content)
    for_test = for_test.drop(["Unnamed: 0"], axis=1)
    status = int(model_rf.predict(for_test))

    if status == 1:
        bot.send_message(user.id, f"Tabrilaymiz🥳, Ushbu kiritigan ma'lumotlar {int(accuracy*100)}% aniqlikda ijobiy chiqdi, va ushbu mijozga kredit berishingiz mumkin!")
    if status == 0:
        bot.send_message(user.id, f"Afsuski☹️, Ushbu kiritigan ma'lumotlar {int(accuracy*100)}% aniqlikda salbiy chiqdi, va ushbu mijozga kredit berish hafli bo'lishi mumkin!")



# Pollingni boshlash
bot.polling(none_stop=True)
